In [268]:
'''
Project V2.0
Contents : Seoul Temperatur
Coding   : jykim, ejlee
Date     : 2023.06.03
Description : 공공데이터 포털 > 기상청_단기예보((구)_동네예보) 조회서비스 이용 오늘의 서울 기온, 눈, 비 정보 가져오기
    https://www.data.go.kr/data/15084084/openapi.do (결과코드 resultCode : 00 성공 그외 '오픈API 에러코드' 참고)
'''
import datetime
import urllib
import requests
import xml.etree.ElementTree as ET

#초단기실황조회 날짜, Base_time(주의: 자정~자정:40분 전까지는 전날짜, 2300 리턴)
def get_date_and_time():
    now = datetime.datetime.now()

    if now.hour == 0 and now.minute < 40:
        current_date = (now - datetime.timedelta(days=1)).strftime("%Y%m%d")
        current_time = "23"
    else:
        current_date = now.strftime("%Y%m%d")
        current_time = now.strftime("%H")            
        
    return current_date, current_time + '00'

#: (초단기) 없음(0), 비(1), 비/눈(2), 눈(3), 빗방울(5), 빗방울눈날림(6), 눈날림(7) 
def get_msg_pty(prPTY=0):
    
    if prPTY == 0 : msg = '없음'
    elif prPTY == 1 : msg = '비'
    elif prPTY == 2 : msg = '비 또는 눈'
    elif prPTY == 3 : msg = '눈'
    elif prPTY == 5 : msg = '빗방울'
    elif prPTY == 6 : msg = '빗방울눈날림'
    else : msg = '눈날림'
        
    return msg

'''
WSD풍속구간(m/s) 통보문 의미 비고
4 미만 /없음/바람이 약하다/ 연기흐름에 따라 풍향감지 가능
4 이상~9 미만 /약간강/바람이 약간 강하다/안면에 감촉을 느끼면서 나뭇잎이 조금 흔들림
9 이상~14 미만 /강/바람이 강하다/나무 가지와 깃발이 가볍게 흔들
14 이상 /매우강/바람이 매우 강하다/먼지가 일고, 작은 나무 전체가 흔들림
'''
def get_msg_wsd(prWSD=0):
    if prWSD < 4 : msg = '없음'
    elif 4 < prWSD < 9 : msg = '바람이 약간 강합니다.'
    elif 9 < prWSD < 14 : msg = '바람이 강합니다.'
    else : msg = '바람이 매우 강합니다.'
    
    return msg

'''
getVilageFcst   단기예보 (fcstValue 예보값)  : 03 TMP PTY WSD SKY => 사용불가
getUltraSrtNcst 초단기실황(obsrValue 실황값) : 00 T1H RN1 WSD 
getUltraSrtFcst 초단기예보(fcstValue 예보값) : 03 T1H RN1 WSD SKY
'''
def api_weather():
    to_day, to_time = get_date_and_time()    #API에 전달할 날짜, 시간 
    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'   
    params ={'serviceKey' : '3Llm9oYomxIM3RbZmB7trrfIA05ylPT4mvj2vnSt3D4KQ2wW3plR0Qx26/LQ8r9hsNEVrSS8v0BXMgy6yccTYQ==', 
             'pageNo' : '1', 
             'numOfRows' : '10', 
             'dataType'  : 'XML', 
             'base_date' : to_day , 
             'base_time' : to_time , #미래시간은 resultCode:03, resultMsg:NO_DATA 리턴.현재시간 에러반횐되는경우 있음
             'nx' : '60', 
             'ny' : '127' }

    response = requests.get(url, params=params)

    root   = ET.fromstring(response.content)  #최상위 요소인 response에 대한 정보 출력 root.tag
    header = root.find('header')              #header 요소에 대한 정보 출력 header.tag
    root   = ET.fromstring(xml_data)          #XML 데이터를 파싱하여 ElementTree 객체 생성

    #resultCode == "00" 이외의 경우 에러처리 추가
    #print("resultCode:", header.find('resultCode').text, "resultMsg:", header.find('resultMsg').text, to_day, to_time)

    for item in root.iter('item'):
        category = item.find('category').text
        if category == 'T1H':  #기온 도
            T1H = item.find('obsrValue').text
        if category == 'RN1':  #1시간 강수량 mm
            RN1 = item.find('obsrValue').text
        if category == 'REH':  #습도
            REH = item.find('obsrValue').text
        if category == 'PTY':  #강수형태
            PTY = item.find('obsrValue').text        
        if category == 'WSD':  #풍속 m/s
            WSD = item.find('obsrValue').text

    return T1H, RN1, REH, PTY, WSD

T1H, RN1, REH, PTY, WSD = api_weather()
print('T1H:', T1H, 'RN1:', RN1, 'REH:', REH, 'PTY:', PTY, 'WSD:', WSD) #스트링으로 리턴
print(get_msg_pty(int(PTY)))
print(get_msg_wsd(float(WSD)))

T1H: 27.3 RN1: 0 REH: 38 PTY: 0 WSD: 4.3
없음
바람이 약간 강합니다.
